# Analyses Descriptives - Qualité de l'Air et Données Socio-Économiques
**Auteur:** Aymeric et Théo  
**Objectif:** Consolider les données urbaines, touristiques et de qualité de l'air pour analyser les corrélations et la représentativité des stations de mesure.

Commençons par importer les bibliothèques principales pour le traitement de données, la visualisation et la modélisation, ainsi que les scripts contenants les fonctions les plus volumineuses. Tous les imports sont faits au début pour les centraliser et ainsi éviter de les oublier ou de les répéter plusieurs fois.

In [22]:
# 1. Configuration et Imports
import pandas as pd
import numpy as np
import warnings
from scripts import preprocessing as prep
from scripts import descriptive_visualization as viz

# Settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
viz.setup_styles()

print("Environnement chargé avec succès.")

Environnement chargé avec succès.


## Partie 1 : Traitement des données Villes & Tourisme
Nous chargeons les données brutes, telles que récupérées sur le site de l'INSEE et de Geodair. Nos données économiques et démographiques (éco-démo dans la suite) sont divisées en 2 datasets, qu'il convient alors de fusionner pour n'en manipuler qu'un seul, contenant toutes les informations sur les villes. Cette fusion des deux datasets requiert d'être vigilant aux noms et types des variables sur lesquelles s'opère la jointure. A ce titre, des conversions de des renommages sont effectués.

Aussi, le champs de notre étude ne porte que sur les villes en France métropolitaine. Ainsi, les villes de Corse et d'outre-mer sont supprimées de notre base de données.

In [23]:
# 2. Chargement et Preprocessing Villes
# Chemins d'accès
path_air = "data/raw_data/data_air_2022.csv"
path_villes = "data/raw_data/data.csv"
path_tourisme = "data/raw_data/BDD_tourisme_communes_2022.csv"

# Chargement initial
df_raw_villes, df_air = prep.load_and_merge_cities(path_air, path_villes, path_tourisme)

# Nettoyage (Renommage, Typage, Filtrage DOM-TOM)
df_villes_clean = prep.process_city_data(df_raw_villes)

# Sauvegarde intermédiaire
df_villes_clean.to_csv("data/processed_data/data_villes_tourisme.csv", index=False)

print(f"Dataset Villes nettoyé : {df_villes_clean.shape}")
df_villes_clean.head()

Dataset Villes nettoyé : (34386, 12)


,code_geo,libelle,part_commerce_transport_services_2023,population_2022,nb_etablissements_2023,densite_population_2022,taux_activite_2022,mediane_niveau_vie_2021,part_industrie_2023,part_construction_2023,nb_hotels_2022,nb_campings_2022
0,1001,L'Abergement-Clémenciat,19.2,859.0,16.0,53.9,82.1,25820.0,0.0,30.8,0.0,0.0
1,1002,L'Abergement-de-Varey,50.0,273.0,4.0,29.8,80.1,24480.0,0.0,0.0,0.0,0.0
2,1004,Ambérieu-en-Bugey,54.6,15554.0,599.0,632.3,76.7,21660.0,7.5,6.0,1.0,0.0
3,1005,Ambérieux-en-Dombes,27.9,1917.0,45.0,120.4,81.3,24610.0,19.8,15.2,2.0,1.0
4,1006,Ambléon,0.0,114.0,1.0,19.4,70.8,24210.0,0.0,0.0,0.0,0.0


## Partie 2 : Analyses Descriptives (Villes)
Commençons par étudier ce dataset contenant des informations sur toutes les villes de France. Cette approche exploratoire nous permettra de construire des éléments de référence pour l'analyse des relevés de pollution. Il s'agit ici de construire des histogrammes pour approximer la distributions de nos variables, ainsi qu'un corrélogramme pour donner un aperçu des relations linéaires qui unissent les variables.

In [24]:
# 3. Visualisation des distributions
cols_to_plot = df_villes_clean.columns[2:] # On exclut seulement code_geo et libelle
viz.plot_distributions(df_villes_clean, cols_to_plot)

print("Graphiques de distribution générés dans 'output/Desc_All_Cities'")

# %%
# 4. Matrice de Corrélation
viz.plot_correlation_heatmap(df_villes_clean)
print("Matrice de corrélation générée.")

Graphiques de distribution générés dans 'output/Desc_All_Cities'
Matrice de corrélation générée.


## Partie 3 : Fusion avec les données Qualité de l'Air (Geodair)
Intégration des relevés de polluants et nettoyage final pour l'étude comparative.

In [17]:
# 5. Chargement Geodair et Merge
geodair = pd.read_csv("data/processed_data/geodair_2022_villes_codgeo2.csv", sep=",")

# Préparation et fusion (gestion des codgeo, types)
df_etude_complete = prep.prepare_geodair_data(geodair, df_villes_clean)

# Sauvegarde du dataset complet
df_etude_complete.to_csv("data/processed_data/data_etude_villes_relevees.csv", index=False, sep=";")

print(f"Dataset Étude Complet : {df_etude_complete.shape}")

Dataset Étude Complet : (2049, 38)


## Partie 4 : Agrégation et Analyse Comparative
Nous regroupons les données par ville et par polluant (moyennes annuelles) et comparons la représentativité des villes ayant une station de mesure par rapport à la France entière.

In [18]:
# 6. Agrégation par Polluant et Ville
df_groupe = prep.aggregate_by_pollutant(df_etude_complete)

polluants = df_groupe["polluant"].unique()
print(f"Polluants identifiés : {polluants}")

Polluants identifiés : ['NOX as NO2' 'O3' 'PM10' 'PM2.5']


In [20]:
# 7. Génération des Datasets par Polluant et Plots Comparatifs
vars_eco = [
    "population_2022", "mediane_niveau_vie_2021", "densite_population_2022",
    "part_industrie_2023", "nb_etablissements_2023", "part_commerce_transport_services_2023"
]

for pol in polluants:
    print(f"--- Traitement : {pol} ---")
    
    # 1. Filtre et Sauvegarde
    df_pol = df_groupe[df_groupe["polluant"] == pol]
    df_pol.to_csv(f"data/processed_data/BDD_par_polluant/dataset_{pol}_final.csv", index=False, sep=';')
    
    # 2. Stats rapides
    med_vie_ech = df_pol['mediane_niveau_vie_2021'].median()
    print(f"   > Médiane Niveau Vie (Echantillon): {med_vie_ech} €")
    
    # 3. Plots Comparatifs (France vs Echantillon)
    viz.plot_comparative_distributions(
        df_sample=df_pol, 
        df_france=df_villes_clean, 
        polluant_name=pol, 
        vars_eco=vars_eco
    )

print("\nAnalyse terminée. Tous les graphiques sont générés.")

--- Traitement : NOX as NO2 ---
   > Médiane Niveau Vie (Echantillon): 21260.0 €
--- Traitement : O3 ---
   > Médiane Niveau Vie (Echantillon): 21585.0 €
--- Traitement : PM10 ---
   > Médiane Niveau Vie (Echantillon): 21280.0 €
--- Traitement : PM2.5 ---
   > Médiane Niveau Vie (Echantillon): 21175.0 €

Analyse terminée. Tous les graphiques sont générés.
